# Tools

- A tool is just a Python function (or API) that is packaged in a way the LLM can understand and call when needed.

![alt text](./Images/tool_types.png)

# Built In Tools

- A built-in tool is a tool that LangChain already provides for you —it’s pre-built, productionready, and requires minimal or no setup. 
- You don’t have to write the function logic yourself — you just import and use i

### Internet Search Tools

In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('top news in india today')

print(results)

Impersonate 'chrome_117' does not exist, using 'random'


India News | Latest India News | Read latest and breaking news from India . Today's top India news headlines, news on Indian politics, elections, government, business, technology, and Bollywood. Stay informed with the latest news today . Get breaking news , live updates, and top headlines from India and around the world on News24 Latest News : Stay updated with the latest news from Mumbai, Delhi, Kolkata, Bengaluru, Hyderabad, Chennai, and across India and the world. Catch breaking news in India Today , top headlines, and live updates from Sports, Cricket, Business, Bollywood, Hollywood, and International news today . Reuters.com is your online source for the latest India news stories and current events, ensuring our readers up to date with any breaking news developments Get Top News of the day with latest top news headlines of today . Get updated with current top news stories from India and the world only on Zee News .


In [2]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


### Shell Tool


In [ ]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('ls')

print(results)

# Custom Tools

- A custom tool is a tool that you define yourself.

## Method 1 : For Creating Tool

In [3]:
from langchain_core.tools import tool

In [ ]:
# Step 1 - create a function

def multiply(a, b):
    return a*b

In [ ]:
# Step 2 - Add DocString

def multiply(a, b):
    """Multiply two numbers"""
    return a*b

In [8]:
# Step 2 - Add type hints

def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [9]:
# Step 3 - Add tool decorator

@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [6]:
result = multiply.invoke({"a":3, "b":5})
result

15

In [ ]:
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.args_schema.model_json_schema()) # This What The LLM See

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


## Method 2 : For Creating Tool - Using StructuredTool

A Structured Tool in LangChain is a special type of tool where the input to the tool follows a structured schema, typically defined using a Pydantic model.

In [17]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field

class MultiplyInput(BaseModel):
    a: int = Field(description="The first number to add")
    b: int = Field(description="The second number to add")
    

def multiply_func(a: int, b: int) -> int:
    return a * b

multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)
print(multiply_tool.args_schema.model_json_schema())

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'title': 'B', 'type': 'integer'}}
{'properties': {'a': {'description': 'The first number to add', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'MultiplyInput', 'type': 'object'}


## Method 3 : For Creating Tool - Using BaseTool Class

- BaseTool is the abstract base class for all tools in LangChain. It defines the core structure and interface that any tool must follow, whether it's a simple one-liner or a fully customized function.

- All other tool types like @tool, StructuredTool are built on top of BaseTool

In [18]:
from langchain.tools import BaseTool
from typing import Type

# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(description="The first number to add")
    b: int = Field(description="The second number to add")
    
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

multiply_tool = MultiplyTool()

result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)
print(multiply_tool.args_schema.model_json_schema())

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'title': 'B', 'type': 'integer'}}
{'properties': {'a': {'description': 'The first number to add', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'MultiplyInput', 'type': 'object'}


# ToolKit

A toolkit is just a collection (bundle) of related tools that serve a common purpose — packaged together for convenience and reusability.

In [19]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [20]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]
    
    
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers
